# 第9章 时序数据

##### 前期预备

In [1]:
import pandas as pd
import numpy as np

## 一、时序的创建
###  1. 四类时间变量
#### 现在理解可能关于③和④有些困惑，后面会作出一些说明

名称 | 描述 | 元素类型 | 创建方式  
:-|:-|:-|:-
① Date times（时间点/时刻） | 描述特定日期或时间点 | Timestamp | to_datetime或date_range
② Time spans（时间段/时期） | 由时间点定义的一段时期 | Period | Period或period_range
③ Date offsets（相对时间差） | 一段时间的相对大小（与夏/冬令时无关） | DateOffset | DateOffset
④ Time deltas（绝对时间差） | 一段时间的绝对大小（与夏/冬令时有关） | Timedelta | to_timedelta或timedelta_range

### 2. 时间点的创建

#### （a）to_datetime方法
#### Pandas在时间点建立的输入格式规定上给了很大的自由度，下面的语句都能正确建立同一时间点

In [2]:
pd.to_datetime('2020.1.1')
pd.to_datetime('2020 1.1')
pd.to_datetime('2020 1 1')
pd.to_datetime('2020 1-1')
pd.to_datetime('2020-1 1')
pd.to_datetime('2020-1-1')
pd.to_datetime('2020/1/1')
pd.to_datetime('1.1.2020')
pd.to_datetime('1.1 2020')
pd.to_datetime('1 1 2020')
pd.to_datetime('1 1-2020')
pd.to_datetime('1-1 2020')
pd.to_datetime('1-1-2020')
pd.to_datetime('1/1/2020')
pd.to_datetime('20200101')
pd.to_datetime('2020.0101')

Timestamp('2020-01-01 00:00:00')

#### 下面的语句都会报错

In [3]:
#pd.to_datetime('2020\\1\\1')
#pd.to_datetime('2020`1`1')
#pd.to_datetime('2020.1 1')
#pd.to_datetime('1 1.2020')

#### 此时可利用format参数强制匹配

In [4]:
pd.to_datetime('2020\\1\\1',format='%Y\\%m\\%d')
pd.to_datetime('2020`1`1',format='%Y`%m`%d')
pd.to_datetime('2020.1 1',format='%Y.%m %d')
pd.to_datetime('1 1.2020',format='%d %m.%Y')

Timestamp('2020-01-01 00:00:00')

#### 同时，使用列表可以将其转为时间点索引

In [5]:
pd.Series(range(2),index=pd.to_datetime(['2020/1/1','2020/1/2']))

2020-01-01    0
2020-01-02    1
dtype: int64

In [6]:
type(pd.to_datetime(['2020/1/1','2020/1/2']))

pandas.core.indexes.datetimes.DatetimeIndex

#### 对于DataFrame而言，如果列已经按照时间顺序排好，则利用to_datetime可自动转换

In [7]:
df = pd.DataFrame({'year': [2020, 2020],'month': [1, 1], 'day': [1, 2]})
pd.to_datetime(df)

0   2020-01-01
1   2020-01-02
dtype: datetime64[ns]

#### （b）时间精度与范围限制
#### 事实上，Timestamp的精度远远不止day，可以最小到纳秒ns

In [8]:
pd.to_datetime('2020/1/1 00:00:00.123456789')

Timestamp('2020-01-01 00:00:00.123456789')

#### 同时，它带来范围的代价就是只有大约584年的时间点是可用的

In [9]:
pd.Timestamp.min

Timestamp('1677-09-21 00:12:43.145225')

In [10]:
pd.Timestamp.max

Timestamp('2262-04-11 23:47:16.854775807')

#### （c）date_range方法
#### 一般来说，start/end/periods（时间点个数）/freq（间隔方法）是该方法最重要的参数，给定了其中的3个，剩下的一个就会被确定

In [11]:
pd.date_range(start='2020/1/1',end='2020/1/10',periods=3)

DatetimeIndex(['2020-01-01 00:00:00', '2020-01-05 12:00:00',
               '2020-01-10 00:00:00'],
              dtype='datetime64[ns]', freq=None)

In [12]:
pd.date_range(start='2020/1/1',end='2020/1/10',freq='D')

DatetimeIndex(['2020-01-01', '2020-01-02', '2020-01-03', '2020-01-04',
               '2020-01-05', '2020-01-06', '2020-01-07', '2020-01-08',
               '2020-01-09', '2020-01-10'],
              dtype='datetime64[ns]', freq='D')

In [13]:
pd.date_range(start='2020/1/1',periods=3,freq='D')

DatetimeIndex(['2020-01-01', '2020-01-02', '2020-01-03'], dtype='datetime64[ns]', freq='D')

In [14]:
pd.date_range(end='2020/1/3',periods=3,freq='D')

DatetimeIndex(['2020-01-01', '2020-01-02', '2020-01-03'], dtype='datetime64[ns]', freq='D')

#### 其中freq参数有许多选项，下面将常用部分罗列如下，更多选项可看[这里](https://pandas.pydata.org/docs/user_guide/timeseries.html#offset-aliases)

符号 | D/B | W | M/Q/Y | BM/BQ/BY | MS/QS/YS | BMS/BQS/BYS | H | T | S
:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:
描述 | 日/工作日 | 周 | 月末 | 月/季/年末日 | 月/季/年末工作日 | 月/季/年初日 | 月/季/年初工作日 | 小时 | 分钟 |秒

In [15]:
pd.date_range(start='2020/1/1',periods=3,freq='T')

DatetimeIndex(['2020-01-01 00:00:00', '2020-01-01 00:01:00',
               '2020-01-01 00:02:00'],
              dtype='datetime64[ns]', freq='T')

In [16]:
pd.date_range(start='2020/1/1',periods=3,freq='M')

DatetimeIndex(['2020-01-31', '2020-02-29', '2020-03-31'], dtype='datetime64[ns]', freq='M')

In [17]:
pd.date_range(start='2020/1/1',periods=3,freq='BYS')

DatetimeIndex(['2020-01-01', '2021-01-01', '2022-01-03'], dtype='datetime64[ns]', freq='BAS-JAN')

#### bdate_range是一个类似与date_range的方法，特点在于可以在自带的工作日间隔设置上，再选择weekmask参数和holidays参数
#### 它的freq中有一个特殊的'C'/'CBM'/'CBMS'选项，表示定制，需要联合weekmask参数和holidays参数使用
#### 例如现在需要将工作日中的周一、周二、周五3天保留，并将部分holidays剔除

In [18]:
weekmask = 'Mon Tue Fri'
holidays = [pd.Timestamp('2020/1/%s'%i) for i in range(7,13)]
#注意holidays
pd.bdate_range(start='2020-1-1',end='2020-1-15',freq='C',weekmask=weekmask,holidays=holidays)

DatetimeIndex(['2020-01-03', '2020-01-06', '2020-01-13', '2020-01-14'], dtype='datetime64[ns]', freq='C')

### 3. DateOffset对象

#### （a）DataOffset与Timedelta的区别
#### Timedelta绝对时间差的特点指无论是冬令时还是夏令时，增减1day都只计算24小时
#### DataOffset相对时间差指，无论一天是23\24\25小时，增减1day都与当天相同的时间保持一致
#### 例如，英国当地时间 2020年03月29日，01:00:00 时钟向前调整 1 小时 变为 2020年03月29日，02:00:00，开始夏令时

In [19]:
ts = pd.Timestamp('2020-3-29 01:00:00', tz='Europe/Helsinki')
ts + pd.Timedelta(days=1)

Timestamp('2020-03-30 02:00:00+0300', tz='Europe/Helsinki')

In [20]:
ts + pd.DateOffset(days=1)

Timestamp('2020-03-30 01:00:00+0300', tz='Europe/Helsinki')

#### 这似乎有些令人头大，但只要把tz（time zone）去除就可以不用管它了，两者保持一致，除非要使用到时区变换

In [21]:
ts = pd.Timestamp('2020-3-29 01:00:00')
ts + pd.Timedelta(days=1)

Timestamp('2020-03-30 01:00:00')

In [22]:
ts + pd.DateOffset(days=1)

Timestamp('2020-03-30 01:00:00')

#### （b）增减一段时间
#### DateOffset的可选参数包括years/months/weeks/days/hours/minutes/seconds

In [23]:
pd.Timestamp('2020-01-01') + pd.DateOffset(minutes=20) - pd.DateOffset(weeks=2)

Timestamp('2019-12-18 00:20:00')

#### （c）各类常用offset对象

freq | D/B | W | (B)M/(B)Q/(B)Y | (B)MS/(B)QS/(B)YS | H | T | S | C |
:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:
offset | DateOffset/BDay | Week | (B)MonthEnd/(B)QuarterEnd/(B)YearEnd | (B)MonthBegin/(B)QuarterBegin/(B)YearBegin | Hour | Minute | Second | CDay(定制工作日)

In [24]:
pd.Timestamp('2020-01-01') + pd.offsets.Week(2)

Timestamp('2020-01-15 00:00:00')

In [25]:
pd.Timestamp('2020-01-01') + pd.offsets.BQuarterBegin(1)

Timestamp('2020-03-02 00:00:00')

#### （d）序列的offset操作
#### 利用apply函数

In [26]:
pd.Series(pd.offsets.BYearBegin(3).apply(i) for i in pd.date_range('20200101',periods=3,freq='Y'))

0   2023-01-02
1   2024-01-01
2   2025-01-01
dtype: datetime64[ns]

#### 直接使用对象加减

In [27]:
pd.date_range('20200101',periods=3,freq='Y') + pd.offsets.BYearBegin(3)

DatetimeIndex(['2023-01-02', '2024-01-01', '2025-01-01'], dtype='datetime64[ns]', freq='A-DEC')

#### 定制offset，可以指定weekmask和holidays参数（思考为什么三个都是一个值）

In [28]:
pd.Series(pd.offsets.CDay(3,weekmask='Wed Fri',holidays='2020010').apply(i)for i in pd.date_range('20200105',periods=3,freq='D'))

0   2020-01-15
1   2020-01-15
2   2020-01-15
dtype: datetime64[ns]

In [29]:
pd.date_range('20200105',periods=3,freq='D')

DatetimeIndex(['2020-01-05', '2020-01-06', '2020-01-07'], dtype='datetime64[ns]', freq='D')

In [30]:
pd.offsets.CDay(3,weekmask='Wed Fri',holidays='2020010')

<3 * CustomBusinessDays>

In [31]:
pd.Series(pd.offsets.CDay(3,weekmask='Wed Fri',holidays='2020010'))

0    <3 * CustomBusinessDays>
dtype: object

## 二、时序的索引及属性
### 1. 索引切片
#### 这一部分几乎与第二章的规则完全一致

In [32]:
rng = pd.date_range('2020','2021', freq='W')
ts = pd.Series(np.random.randn(len(rng)), index=rng)
ts.head()

2020-01-05   -0.931308
2020-01-12   -1.532354
2020-01-19   -1.476239
2020-01-26    0.176536
2020-02-02    0.420441
Freq: W-SUN, dtype: float64

In [33]:
ts['2020-01-26']

0.17653598183473693

#### 合法字符自动转换为时间点

In [34]:
ts['2020-01-26':'20200726']

2020-01-26    0.176536
2020-02-02    0.420441
2020-02-09    1.746537
2020-02-16    0.102323
2020-02-23    0.386403
2020-03-01    0.566397
2020-03-08    1.123252
2020-03-15   -0.081908
2020-03-22    1.383383
2020-03-29   -0.989921
2020-04-05    1.331185
2020-04-12    0.296048
2020-04-19    0.626084
2020-04-26    0.259681
2020-05-03   -0.841465
2020-05-10    0.581026
2020-05-17   -0.927517
2020-05-24    0.350076
2020-05-31    0.763075
2020-06-07    2.198205
2020-06-14    0.041841
2020-06-21    1.302279
2020-06-28   -0.985542
2020-07-05   -1.098149
2020-07-12   -0.466304
2020-07-19   -0.947272
2020-07-26   -0.231524
Freq: W-SUN, dtype: float64

### 2. 子集索引

In [35]:
ts['2020-7'].head()

2020-07-05   -1.098149
2020-07-12   -0.466304
2020-07-19   -0.947272
2020-07-26   -0.231524
Freq: W-SUN, dtype: float64

#### 支持混合形态索引

In [36]:
ts['2011-1':'20200726']

2020-01-05   -0.931308
2020-01-12   -1.532354
2020-01-19   -1.476239
2020-01-26    0.176536
2020-02-02    0.420441
2020-02-09    1.746537
2020-02-16    0.102323
2020-02-23    0.386403
2020-03-01    0.566397
2020-03-08    1.123252
2020-03-15   -0.081908
2020-03-22    1.383383
2020-03-29   -0.989921
2020-04-05    1.331185
2020-04-12    0.296048
2020-04-19    0.626084
2020-04-26    0.259681
2020-05-03   -0.841465
2020-05-10    0.581026
2020-05-17   -0.927517
2020-05-24    0.350076
2020-05-31    0.763075
2020-06-07    2.198205
2020-06-14    0.041841
2020-06-21    1.302279
2020-06-28   -0.985542
2020-07-05   -1.098149
2020-07-12   -0.466304
2020-07-19   -0.947272
2020-07-26   -0.231524
Freq: W-SUN, dtype: float64

### 3. 时间点的属性

#### 采用dt对象可以轻松获得关于时间的信息

In [37]:
pd.Series(ts.index).dt.week.head()

0    1
1    2
2    3
3    4
4    5
dtype: int64

In [38]:
pd.Series(ts.index).dt.day.head()

0     5
1    12
2    19
3    26
4     2
dtype: int64

#### 利用strftime可重新修改时间格式

In [39]:
pd.Series(ts.index).dt.strftime('%Y-间隔1-%m-间隔2-%d').head()

0    2020-01-05 00:00:00
1    2020-01-12 00:00:00
2    2020-01-19 00:00:00
3    2020-01-26 00:00:00
4    2020-02-02 00:00:00
dtype: object

#### 对于datetime对象可以直接通过属性获取信息

In [40]:
pd.date_range('2020','2021', freq='W').month

Int64Index([ 1,  1,  1,  1,  2,  2,  2,  2,  3,  3,  3,  3,  3,  4,  4,  4,  4,
             5,  5,  5,  5,  5,  6,  6,  6,  6,  7,  7,  7,  7,  8,  8,  8,  8,
             8,  9,  9,  9,  9, 10, 10, 10, 10, 11, 11, 11, 11, 11, 12, 12, 12,
            12],
           dtype='int64')

In [41]:
pd.date_range('2020','2021', freq='W').weekday

Int64Index([6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
            6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
            6, 6, 6, 6, 6, 6, 6, 6],
           dtype='int64')

## 三、重采样

#### 所谓重采样，就是指resample函数，它可以看做时序版本的groupby函数

### 1. resample对象的基本操作
#### 采样频率一般设置为上面提到的offset字符

In [42]:
df_r = pd.DataFrame(np.random.randn(1000, 3),index=pd.date_range('1/1/2020', freq='S', periods=1000),columns=['A', 'B', 'C'])
df_r

,A,B,C
2020-01-01 00:00:00,1.645806,0.159110,-0.948985
2020-01-01 00:00:01,0.855087,-0.488582,-0.648831
2020-01-01 00:00:02,0.921230,0.607214,-1.911679
2020-01-01 00:00:03,1.436407,1.688003,1.438719
2020-01-01 00:00:04,0.484013,-1.426008,-0.651722
...,...,...,...
2020-01-01 00:16:35,-2.691530,-0.651270,-0.129745
2020-01-01 00:16:36,-0.427126,-0.190155,0.664936
2020-01-01 00:16:37,1.505168,1.467505,0.650764
2020-01-01 00:16:38,-1.628272,-0.419288,-0.304762


In [43]:
r = df_r.resample('3min')
r

In [44]:
r.sum()

,A,B,C
2020-01-01 00:00:00,9.693988,-3.135939,0.239510
2020-01-01 00:03:00,-16.567185,2.783230,3.862348
2020-01-01 00:06:00,-16.383425,-2.984832,-1.471107
2020-01-01 00:09:00,-14.003082,-1.735100,-1.387657
2020-01-01 00:12:00,13.878552,-15.307484,-4.130727
2020-01-01 00:15:00,10.287824,-0.938712,-15.312904


同理：

In [45]:
df_r2 = pd.DataFrame(np.random.randn(200, 3),index=pd.date_range('1/1/2020', freq='D', periods=200),columns=['A', 'B', 'C'])
df_r2

,A,B,C
2020-01-01,-0.112520,0.836395,-1.335562
2020-01-02,1.332855,-0.555328,-0.878985
2020-01-03,-0.471619,-1.250608,-1.348192
2020-01-04,1.229221,-0.131250,-3.103605
2020-01-05,1.931113,-0.628133,0.377643
...,...,...,...
2020-07-14,-0.284826,1.110605,0.792571
2020-07-15,0.721619,-0.204963,1.297406
2020-07-16,1.838089,-1.329827,0.042690
2020-07-17,2.063511,-2.628187,-0.052753


In [46]:
r = df_r2.resample('CBMS')
r.sum()

,A,B,C
2020-01-01,8.439522,-0.080663,-1.856235
2020-02-03,-7.229304,6.345693,0.999188
2020-03-02,3.859069,0.294616,0.226827
2020-04-01,4.115339,-6.229292,-2.077723
2020-05-01,-4.614431,-2.482882,-0.374926
2020-06-01,-0.843533,6.162359,-3.219478
2020-07-01,3.432178,-7.683135,-3.460264


### 2. 采样聚合

In [47]:
r = df_r.resample('3T')
r

In [48]:
r['A'].mean()

2020-01-01 00:00:00    0.053855
2020-01-01 00:03:00   -0.092040
2020-01-01 00:06:00   -0.091019
2020-01-01 00:09:00   -0.077795
2020-01-01 00:12:00    0.077103
2020-01-01 00:15:00    0.102878
Freq: 3T, Name: A, dtype: float64

In [49]:
r['A'].agg([np.sum, np.mean, np.std])

,sum,mean,std
2020-01-01 00:00:00,9.693988,0.053855,0.897895
2020-01-01 00:03:00,-16.567185,-0.092040,1.042141
2020-01-01 00:06:00,-16.383425,-0.091019,0.951917
2020-01-01 00:09:00,-14.003082,-0.077795,1.002560
2020-01-01 00:12:00,13.878552,0.077103,1.052236
2020-01-01 00:15:00,10.287824,0.102878,1.101983


#### 类似地，可以使用函数/lambda表达式

In [50]:
r.agg({'A': np.sum,'B': lambda x: max(x)-min(x)})

,A,B
2020-01-01 00:00:00,9.693988,5.261584
2020-01-01 00:03:00,-16.567185,5.741908
2020-01-01 00:06:00,-16.383425,5.505325
2020-01-01 00:09:00,-14.003082,5.035620
2020-01-01 00:12:00,13.878552,5.560066
2020-01-01 00:15:00,10.287824,4.197209


### 3. 采样组的迭代
#### 采样组的迭代和groupby迭代完全类似，对于每一个组都可以分别做相应操作

In [51]:
small = pd.Series(range(6),index=pd.to_datetime(['2020-01-01 00:00:00', '2020-01-01 00:30:00'
                                                 , '2020-01-01 00:31:00','2020-01-01 01:00:00'
                                                 ,'2020-01-01 03:00:00','2020-01-01 03:05:00']))
resampled = small.resample('H')
for name, group in resampled:
    print("Group: ", name)
    print("-" * 27)
    print(group, end="\n\n")

Group:  2020-01-01 00:00:00
---------------------------
2020-01-01 00:00:00    0
2020-01-01 00:30:00    1
2020-01-01 00:31:00    2
dtype: int64

Group:  2020-01-01 01:00:00
---------------------------
2020-01-01 01:00:00    3
dtype: int64

Group:  2020-01-01 02:00:00
---------------------------
Series([], dtype: int64)

Group:  2020-01-01 03:00:00
---------------------------
2020-01-01 03:00:00    4
2020-01-01 03:05:00    5
dtype: int64



## 四、窗口函数

#### 下面主要介绍pandas中两类主要的窗口(window)函数:rolling/expanding

In [52]:
s = pd.Series(np.random.randn(1000),index=pd.date_range('1/1/2020', periods=1000))
s.head()

2020-01-01    0.435153
2020-01-02    0.179944
2020-01-03   -0.180207
2020-01-04    1.181325
2020-01-05    0.196201
Freq: D, dtype: float64

### 1. Rolling
#### （a）常用聚合
#### 所谓rolling方法，就是规定一个窗口，它和groupby对象一样，本身不会进行操作，需要配合聚合函数才能计算结果

In [53]:
s.rolling(window=50)

Rolling [window=50,center=False,axis=0]

In [54]:
s.rolling(window=50).mean()

2020-01-01         NaN
2020-01-02         NaN
2020-01-03         NaN
2020-01-04         NaN
2020-01-05         NaN
                ...   
2022-09-22   -0.359656
2022-09-23   -0.337889
2022-09-24   -0.323838
2022-09-25   -0.331541
2022-09-26   -0.325506
Freq: D, Length: 1000, dtype: float64

#### min_periods参数是指需要的非缺失数据点数量阀值

In [55]:
s.rolling(window=50,min_periods=3).mean().head()

2020-01-01         NaN
2020-01-02         NaN
2020-01-03    0.144963
2020-01-04    0.404054
2020-01-05    0.362483
Freq: D, dtype: float64

#### count/sum/mean/median/min/max/std/var/skew/kurt/quantile/cov/corr都是常用的聚合函数
#### （b）rolling的apply聚合
#### 使用apply聚合时，只需记住传入的是window大小的Series，输出的必须是标量即可，比如如下计算变异系数

In [56]:
s.rolling(window=50,min_periods=3).apply(lambda x:x.std()/x.mean()).head()

2020-01-01         NaN
2020-01-02         NaN
2020-01-03    2.132733
2020-01-04    1.426538
2020-01-05    1.400772
Freq: D, dtype: float64

#### （c）基于时间的rolling

In [57]:
s.rolling('15D').mean().head()

2020-01-01    0.435153
2020-01-02    0.307548
2020-01-03    0.144963
2020-01-04    0.404054
2020-01-05    0.362483
Freq: D, dtype: float64

#### 可选closed='right'（默认）\'left'\'both'\'neither'参数，决定端点的包含情况

In [58]:
s.rolling('15D', closed='right').sum().head()

2020-01-01    0.435153
2020-01-02    0.615097
2020-01-03    0.434890
2020-01-04    1.616215
2020-01-05    1.812416
Freq: D, dtype: float64

### 2. Expanding

#### （a）expanding函数
#### 普通的expanding函数等价与rolling(window=len(s),min_periods=1)，是对序列的累计计算

In [59]:
s.rolling(window=len(s),min_periods=1).sum().head()

2020-01-01    0.435153
2020-01-02    0.615097
2020-01-03    0.434890
2020-01-04    1.616215
2020-01-05    1.812416
Freq: D, dtype: float64

In [60]:
s.expanding().sum().head()

2020-01-01    0.435153
2020-01-02    0.615097
2020-01-03    0.434890
2020-01-04    1.616215
2020-01-05    1.812416
Freq: D, dtype: float64

#### apply方法也是同样可用的

In [61]:
s.expanding().apply(lambda x:sum(x)).head()

2020-01-01    0.435153
2020-01-02    0.615097
2020-01-03    0.434890
2020-01-04    1.616215
2020-01-05    1.812416
Freq: D, dtype: float64

#### （b）几个特别的Expanding类型函数
#### cumsum/cumprod/cummax/cummin都是特殊expanding累计计算方法

In [62]:
s.cumsum().head()

2020-01-01    0.435153
2020-01-02    0.615097
2020-01-03    0.434890
2020-01-04    1.616215
2020-01-05    1.812416
Freq: D, dtype: float64

In [63]:
s.cumprod().head()

2020-01-01    0.435153
2020-01-02    0.078303
2020-01-03   -0.014111
2020-01-04   -0.016669
2020-01-05   -0.003271
Freq: D, dtype: float64

In [64]:
s.cummax().head()

2020-01-01    0.435153
2020-01-02    0.435153
2020-01-03    0.435153
2020-01-04    1.181325
2020-01-05    1.181325
Freq: D, dtype: float64

In [65]:
s.cummin().head()

2020-01-01    0.435153
2020-01-02    0.179944
2020-01-03   -0.180207
2020-01-04   -0.180207
2020-01-05   -0.180207
Freq: D, dtype: float64

#### shift/diff/pct_change都是涉及到了元素关系
#### ①shift是指序列索引不变，但值向后移动
#### ②diff是指前后元素的差，period参数表示间隔，默认为1，并且可以为负
#### ③pct_change是值前后元素的变化百分比，period参数与diff类似

In [66]:
s.shift(2).head()

2020-01-01         NaN
2020-01-02         NaN
2020-01-03    0.435153
2020-01-04    0.179944
2020-01-05   -0.180207
Freq: D, dtype: float64

In [67]:
s.diff(3).head()

2020-01-01         NaN
2020-01-02         NaN
2020-01-03         NaN
2020-01-04    0.746172
2020-01-05    0.016257
Freq: D, dtype: float64

In [68]:
s.pct_change(3).head()

2020-01-01         NaN
2020-01-02         NaN
2020-01-03         NaN
2020-01-04    1.714734
2020-01-05    0.090345
Freq: D, dtype: float64

## 五、问题与练习

### 1、问题

#### 【问题一】 如何对date_range进行批量加帧操作或对某一时间段加大时间戳密度？

###### 通过apply方法

In [69]:
pd.Series(pd.offsets.BMonthBegin(2).apply(i) for i in pd.date_range('20200101',periods=4,freq='M'))

0   2020-03-02
1   2020-04-01
2   2020-05-01
3   2020-06-01
dtype: datetime64[ns]

#### 【问题二】 如何批量增加TimeStamp的精度？

In [70]:
pd.Timestamp('20200101')

Timestamp('2020-01-01 00:00:00')

In [71]:
pd.Series(pd.offsets.Second(1).apply(i) for i in pd.date_range('20200101',periods=4,freq='M'))

0   2020-01-31 00:00:01
1   2020-02-29 00:00:01
2   2020-03-31 00:00:01
3   2020-04-30 00:00:01
dtype: datetime64[ns]

#### 【问题三】 对于超出处理时间的时间点，是否真的完全没有处理方法？

超出处理时间的时间点可以使用apply方法对时间进行截断，或者使用其他的值来代替。

#### 【问题四】 给定一组非连续的日期，怎么快速找出位于其最大日期和最小日期之间，且没有出现在该组日期中的日期？

In [72]:
pd.date_range('20200101',periods=4,freq='D').min()

Timestamp('2020-01-01 00:00:00', freq='D')

In [73]:
dt = pd.to_datetime(['20200101','20200103','20200106','20200110'])
pd.Series(pd.date_range(start=dt.min(),end=dt.max(),freq='D')).apply(lambda x:x if x not in dt else None).dropna()

1   2020-01-02
3   2020-01-04
4   2020-01-05
6   2020-01-07
7   2020-01-08
8   2020-01-09
dtype: datetime64[ns]

### 2、练习

#### 【练习一】 现有一份关于某超市牛奶销售额的时间序列数据，请完成下列问题：

In [74]:
pd.read_csv('joyful-pandas-master/data/time_series_one.csv').head()

,日期,销售额
0,2017/2/17,2154
1,2017/2/18,2095
2,2017/2/19,3459
3,2017/2/20,2198
4,2017/2/21,2413


#### （a）销售额出现最大值的是星期几？（提示：利用dayofweek函数）

In [75]:
df = pd.read_csv('joyful-pandas-master/data/time_series_one.csv')
df.head()

,日期,销售额
0,2017/2/17,2154
1,2017/2/18,2095
2,2017/2/19,3459
3,2017/2/20,2198
4,2017/2/21,2413


In [76]:
df['日期'] = pd.to_datetime(df['日期'])
df['日期'].dt.dayofweek[df['销售额'].idxmax()]

6

#### （b）计算除去春节、国庆、五一节假日的月度销售总额

In [77]:
holiday = pd.date_range(start='20170501', end='20170503').append(
          pd.date_range(start='20171001', end='20171007')).append(
          pd.date_range(start='20180215', end='20180221')).append(
          pd.date_range(start='20180501', end='20180503')).append(
          pd.date_range(start='20181001', end='20181007')).append(
          pd.date_range(start='20190204', end='20190224')).append(
          pd.date_range(start='20190501', end='20190503')).append(
          pd.date_range(start='20191001', end='20191007'))
result = df[~df['日期'].isin(holiday)].set_index('日期').resample('MS').sum()
result.head()

,销售额
日期,
2017-02-01,31740
2017-03-01,80000
2017-04-01,74734
2017-05-01,76237
2017-06-01,80750


#### （c）按季度计算周末（周六和周日）的销量总额

In [78]:
result = df[df['日期'].dt.dayofweek.isin([5,6])].set_index('日期').resample('QS').sum()
result.head()

,销售额
日期,
2017-01-01,32894
2017-04-01,66692
2017-07-01,69099
2017-10-01,70384
2018-01-01,74671


#### （d）从最后一天开始算起，跳过周六和周一，以5天为一个时间单位向前计算销售总和

In [79]:
df_temp = df[~df['日期'].dt.dayofweek.isin([5,6])].set_index('日期').iloc[::-1]
L_temp,date_temp = [],[0]*df_temp.shape[0]
for i in range(df_temp.shape[0]//5):
    L_temp.extend([i]*5)
L_temp.extend([df_temp.shape[0]//5]*(df_temp.shape[0]-df_temp.shape[0]//5*5))
date_temp = pd.Series([i%5==0 for i in range(df_temp.shape[0])])
df_temp['num'] = L_temp
result = pd.DataFrame({'5天总额':df_temp.groupby('num')['销售额'].sum().values},
                       index=df_temp.reset_index()[date_temp]['日期']).iloc[::-1]
result.head()

,5天总额
日期,
2017-02-22,9855
2017-03-01,12296
2017-03-08,13323
2017-03-15,13845
2017-03-22,11356


#### （e）假设现在发现数据有误，所有同一周里的周一与周五的销售额记录颠倒了，请计算2018年中每月第一个周一的销售额（如果该周没有周一或周五的记录就保持不动）

In [80]:
from datetime import datetime 
df_temp = df.copy()
df_fri = df.shift(4)[df.shift(4)['日期'].dt.dayofweek==1]['销售额']
df_mon = df.shift(-4)[df.shift(-4)['日期'].dt.dayofweek==5]['销售额']
df_temp.loc[df_fri.index,['销售额']] = df_fri
df_temp.loc[df_mon.index,['销售额']] = df_mon
df_temp.loc[df_temp[df_temp['日期'].dt.year==2018]['日期'][
        df_temp[df_temp['日期'].dt.year==2018]['日期'].apply(
        lambda x:True if datetime.strptime(str(x).split()[0],'%Y-%m-%d').weekday() == 0 
        and 1 <= datetime.strptime(str(x).split()[0],'%Y-%m-%d').day <= 7 else False)].index,:]

,日期,销售额
318,2018-01-01,2863.0
353,2018-02-05,2321.0
381,2018-03-05,2705.0
409,2018-04-02,2487.0
444,2018-05-07,3204.0
472,2018-06-04,2927.0
500,2018-07-02,2574.0
535,2018-08-06,2504.0
563,2018-09-03,2483.0
591,2018-10-01,2431.0


#### 【练习二】 继续使用上一题的数据，请完成下列问题：

#### （a）以50天为窗口计算滑窗均值和滑窗最大值（min_periods设为1）

In [81]:
df = pd.read_csv('joyful-pandas-master/data/time_series_one.csv',index_col='日期',parse_dates=['日期'])
df['销售额'].rolling(window=50,min_periods=1).mean().head()

日期
2017-02-17    2154.000000
2017-02-18    2124.500000
2017-02-19    2569.333333
2017-02-20    2476.500000
2017-02-21    2463.800000
Name: 销售额, dtype: float64

In [82]:
df['销售额'].rolling(window=50,min_periods=1).max().head()

日期
2017-02-17    2154.0
2017-02-18    2154.0
2017-02-19    3459.0
2017-02-20    3459.0
2017-02-21    3459.0
Name: 销售额, dtype: float64

#### （b）现在有如下规则：若当天销售额超过向前5天的均值，则记为1，否则记为0，请给出2018年相应的计算结果

In [83]:
def f(x):
    if len(x) == 6:
        return 1 if x[-1]>np.mean(x[:-1]) else 0
    else:
        return 0
result_b = df.loc[pd.date_range(start='20171227',end='20181231'),:].rolling(
                                                    window=6,min_periods=1).agg(f)[5:].head()
result_b.head()

,销售额
2018-01-01,1.0
2018-01-02,0.0
2018-01-03,0.0
2018-01-04,0.0
2018-01-05,0.0


#### （c）将(c)中的“向前5天”改为“向前非周末5天”，请再次计算结果

In [84]:
def f(x):
    if len(x) == 8:
        return 1 if x[-1]>np.mean(x[:-1][pd.Series([
            False if i in [5,6] else True for i in x[:-1].index.dayofweek],index=x[:-1].index)]) else 0
    else:
        return 0
result_c = df.loc[pd.date_range(start='20171225',end='20181231'),:].rolling(
                                    window=8,min_periods=1).agg(f)[7:].head()
result_c.head()

,销售额
2018-01-01,1.0
2018-01-02,0.0
2018-01-03,0.0
2018-01-04,0.0
2018-01-05,0.0
